## Módulo: Analytics Engineering
    
## Aula 2 - Exercício 1

#### Preparar um pipeline com as camadas "bronze", "silver" e "gold" para o dataset sobre informações de clientes em um banco. O dataset está disponível para download no seguinte website: "https://www.kaggle.com/datasets/parisrohan/credit-score-classification".

#### Efetuar a limpeza de dados necessária e garantir a qualidade de dados com todos os conceitos apresentados até agora.

#### Armazenar todas as camadas em um banco de dados no PostgreSQL.

#### Dicionário de Dados

- **Age (Idade):** Representa a idade da pessoa.
- **Annual_Income (Renda Anual):** Representa a renda anual da pessoa.
- **Monthly_Inhand_Salary (Salário Mensal Líquido):** Representa o salário base mensal de uma pessoa.
- **Num_Bank_Accounts (Número de Contas Bancárias):** Representa o número de contas bancárias que uma pessoa possui.
- **Num_Credit_Card (Número de Cartões de Crédito):** Representa o número de outros cartões de crédito que uma pessoa possui.
- **Interest_Rate (Taxa de Juros):** Representa a taxa de juros no cartão de crédito (porcentagem).
- **Num_of_Loan (Número de Empréstimos):** Representa o número de empréstimos obtidos no banco.
- **Delay_from_due_date (Atraso a partir da Data de Vencimento):** Representa o número médio de dias de atraso a partir da data de pagamento (dias).
- **Num_of_Delayed_Payment (Número de Pagamentos Atrasados):** Representa o número médio de pagamentos atrasados por uma pessoa.
- **Changed_Credit_Limit (Alteração no Limite de Crédito):** Representa a alteração percentual no limite de crédito do cartão (porcentagem).
- **Num_Credit_Inquiries (Número de Consultas de Crédito):** Representa o número de consultas de cartão de crédito.
- **Credit_Mix (Mix de Crédito):** Representa a classificação da mistura de créditos (Ruim, Padrão, Bom).
- **Outstanding_Debt (Dívida Pendente):** Representa a dívida restante a ser paga.
- **Credit_Utilization_Ratio (Taxa de Utilização de Crédito):** Representa a taxa de utilização do cartão de crédito (porcentagem).
- **Credit_History_Age (Idade da História de Crédito):** Representa a idade da história de crédito da pessoa (dias).
- **Payment_of_Min_Amount (Pagamento do Valor Mínimo):** Representa se apenas o valor mínimo foi pago pela pessoa.
- **Total_EMI_per_month (Total de EMI Mensal):** Representa os pagamentos mensais de EMI.
- **Amount_invested_monthly (Valor Investido Mensalmente):** Representa o valor investido mensalmente pelo cliente.
- **Monthly_Balance (Saldo Mensal):** Representa o valor do saldo mensal do cliente.

In [36]:
import pandas as pd
import numpy as np

In [ ]:
# Supondo que o DataFrame df já está carregado com os dados acima


df = pd.read_csv("dados/test.csv")
df.head(3)

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report") #cria o relatório

profile.to_file("resultados.html") #salva os resultados em um arquivo

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [57]:
df_cln = df.copy()

In [ ]:
df_cln

In [ ]:
# Preencher valores NaN para dados numéricos com a média ou mediana (exemplo: Monthly_Inhand_Salary)
df_cln['Monthly_Inhand_Salary'].fillna(df['Monthly_Inhand_Salary'].median())

# Para dados categóricos, considerar preenchimento com a moda ou uma categoria específica
df_cln['Credit_History_Age'].fillna('Unknown')

In [ ]:
# Converter colunas de idade, e outras colunas numéricas de object para int ou float
df_cln['Age'] = pd.to_numeric(df['Age'], errors='coerce')  # Converte e trata erros

# Remover ou tratar linhas com valores absurdos ou claramente errados (exemplo: Age = 4975)
df_cln = df_cln[ df_cln['Age'] <= 100]  # Supondo que a idade máxima razoável seja 100
df_cln = df_cln[ df_cln['Age'] >= 1]
# Converter datas de meses para tipo datetime se necessário
df_cln['Month'] = pd.to_datetime(df_cln['Month'], format='%B', errors='coerce').dt.month

# Corrigir e padronizar strings (exemplo: remoção de caracteres não desejados nos nomes)
df_cln['Name'] = df_cln['Name'].str.replace(r'[^\w\s]', '', regex=True)
df_cln['Occupation'] = df_cln['Occupation'].str.replace('_', 'Unknown')

In [63]:
# Corrigir SSN e outros IDs que possam ter sido mal formatados
df_cln['SSN'] = df_cln['SSN'].str.replace('-', '')

# Tratar dados específicos que estejam incorretos
df_cln['Payment_of_Min_Amount'] = df_cln['Payment_of_Min_Amount'].replace({'NM': 'No', 'Yes': 'Yes', 'No': 'No'})

# Converter 'Credit_History_Age' para um formato numérico de meses totais para facilitar análises
def convert_credit_age_to_months(age_str):
    if pd.isnull(age_str) or age_str == 'Unknown':
        return np.nan
    years, months = age_str.split(' Years and ')
    years = int(years)
    months = int(months.split(' Months')[0])
    return years * 12 + months

df_cln['Credit_History_Age'] = df_cln['Credit_History_Age'].apply(convert_credit_age_to_months)


In [ ]:

# Remover duplicatas, se houver
df_cln.drop_duplicates(inplace=True)

# Revisar as colunas e tipos de dados
print(df_cln.info())

In [ ]:
df_cln.dtypes

In [ ]:
df.describe()

In [ ]:
df_cln

In [ ]:
# Corrigindo a idade
# df['Age'] = df['Age'].apply(lambda x: np.nan if x < 0 or x > 100 else x)

# Imputando salários faltantes
median_salary = df_cln['Monthly_Inhand_Salary'].median()
df_cln['Monthly_Inhand_Salary'].fillna(median_salary, inplace=True)

# Corrigindo o número de contas bancárias
df_cln['Num_Bank_Accounts'] = df_cln['Num_Bank_Accounts'].apply(lambda x: np.nan if x < 0 else x)

# Investigando valores altos de cartões de crédito
max_credit_cards = df_cln['Num_Credit_Card'].quantile(0.99)  # Exemplo: Usar o percentil 99 como corte
df_cln['Num_Credit_Card'] = df_cln['Num_Credit_Card'].apply(lambda x: np.nan if x > max_credit_cards else x)

# Corrigindo taxas de juros
df_cln['Interest_Rate'] = df_cln['Interest_Rate'].apply(lambda x: np.nan if x > 100 else x)  #

In [ ]:
df_cln

In [77]:
# Removendo valores negativos de atrasos
df_cln['Delay_from_due_date'] = df_cln['Delay_from_due_date'].apply(lambda x: 0 if x < 0 else x)

# Verificando e corrigindo consultas de crédito
max_inquiries = df_cln['Num_Credit_Inquiries'].quantile(0.99)
df_cln['Num_Credit_Inquiries'] = df_cln['Num_Credit_Inquiries'].apply(lambda x: np.nan if x > max_inquiries else x)

# Verificação e correção do EMI mensal
max_emi = df_cln['Total_EMI_per_month'].quantile(0.99)
df_cln['Total_EMI_per_month'] = df_cln['Total_EMI_per_month'].apply(lambda x: np.nan if x > max_emi else x)


In [ ]:
# Exibindo informações atualizadas
print(df_cln.describe())

In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df_cln, title="Pandas Profiling Report") #cria o relatório

profile.to_file("novos_resultados.html") #salva os resultados em um arquivo

In [ ]:
df_cln['ID'] = df_cln['ID'].apply(lambda x: int(x, 16))

# Limpeza na coluna SSN para remover caracteres não-numéricos
df_cln['SSN'] = df_cln['SSN'].str.replace(r'\D', '', regex=True)

# Corrigir a coluna Occupation onde dados foram concatenados indevidamente
df_cln['Occupation'] = df_cln['Occupation'].replace(r'UnknownUnknownUnknownUnknownUnknownUnknownUnknown', 'Unknown', regex=True)

In [ ]:
# Removendo a coluna Month se não for possível imputar dados significativos
df_cln.drop('Month', axis=1, inplace=True)

In [ ]:
# Converter valores de Amount_invested_monthly para numérico, tratando erros
df_cln['Amount_invested_monthly'] = pd.to_numeric(df_cln['Amount_invested_monthly'].str.replace(r'[^0-9\.]', '', regex=True), errors='coerce')


In [ ]:
# Substituir '_' por um valor nulo ou outro valor categórico relevante em Credit_Mix
df_cln['Credit_Mix'] = df_cln['Credit_Mix'].replace('_', np.nan)


In [ ]:
# Imputar Monthly_Inhand_Salary faltantes com a mediana
df_cln['Monthly_Inhand_Salary'].fillna(df_cln['Monthly_Inhand_Salary'].median(), inplace=True)


In [ ]:
# Verificar os tipos de dados e ajustar conforme necessário
df_cln.info()

# Verificar se há mais valores ausentes ou duplicatas
print(df_cln.isnull().sum())
df_cln.drop_duplicates(inplace=True)


In [ ]:
# Visualizar as primeiras linhas do DataFrame limpo
print(df_cln.head())
